In [1]:
import os
import random
import torch
import numpy as np
import pickle as pkl
from analysis import *
import argparse
from sys import platform

In [2]:
torch.manual_seed(12)
torch.cuda.manual_seed(12)
np.random.seed(12)
torch.backends.cudnn.deterministics = True
torch.set_num_threads(1)

In [3]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())

print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))

True
1
0
NVIDIA TITAN V


In [4]:
# iv: image option
length = 512
channel = 96
min_CNN = 200
n_classes = 40
classes = range(n_classes)


In [5]:
if platform == "linux" or platform == "linux2":
    torch_models_dir = r"/media/titan/AI Research/Data/CVPR2021-02785/CVPR2021-02785/preprocessed/torch_models"
elif platform == "win32":
    torch_models_dir = r"D:\Data\CVPR2021-02785\CVPR2021-02785\preprocessed\torch_models"
eeg_dataset, splits_path, splits_shuffled_path = os.listdir(torch_models_dir)

In [6]:
eeg_dataset = os.path.join(torch_models_dir, eeg_dataset)
splits_path = os.path.join(torch_models_dir, splits_path)
# splits_path = os.path.join(torch_models_dir, splits_shuffled_path)
print(eeg_dataset,'\n', splits_path)

/media/titan/AI Research/Data/CVPR2021-02785/CVPR2021-02785/preprocessed/torch_models/imagenet40-1000-1.pth 
 /media/titan/AI Research/Data/CVPR2021-02785/CVPR2021-02785/preprocessed/torch_models/imagenet40-1000-1_splits.pth


In [7]:
opt = {
    # Dataset options
    "iv": "image",
    "offset": None,
    "results_file": "results.pkl",
    "subject": 0,
    "run": "none",
    "eeg_dataset": eeg_dataset,
    "splits_path": splits_path,
    "fold": 5,
    #Training options
    "batch_size": 16,
    "optim": "Adam",
    "learning_rate": 0.001,
    "learning_rate_decay_by": 0.5,
    "learning_rate_decay_every": 10,
    "epochs": 100,
    "GPUindex": 0,
    "kind":"incremental",
    #Backend options
    "no_cuda": False,
    "classifier": None
}
opt = argparse.Namespace(**opt)
print(opt.iv)

image


In [8]:
%load_ext autoreload
%autoreload 2
from torch.utils.data import DataLoader
from data_loader import EEGDataset, Splitter, SplitterWithData
from EEG_Encoder.LSTM import classifier_LSTM
from EEG_Encoder.CNN import classifier_CNN
from EEG_Encoder.EEGNet import classifier_EEGNet
from EEG_Encoder.SyncNet import classifier_SyncNet
from EEG_Encoder.EEGChannelNet import classifier_EEGChannelNet
from EEG_Encoder.net_generator import Classifier
from EEG_Encoder.net_trainer import net_trainer
from p_values import *
from torchinfo import summary

In [9]:
def load_dataset(iv,
             offset,
             eeg_dataset,
             splits_path,
             split_num, # (0-4) - 5 fold cross validation
             total,
             classes,
             classifier,
             batch_size,
             GPUindex,
             length, # 512
             channel, # 96
             min_CNN,
             opt,
             kind):        
    # Load dataset
    dataset = EEGDataset(iv, eeg_dataset, classifier, map_idx = None)
    print("DONE: LOAD DATASET")
    # Create loaders for LSTM/MLP/CNN/SCNN/EEGNet/SyncNet/EEGChannelNet
    if kind=="from-scratch":
        relabel = False
    if kind=="incremental":
        relabel = False
    if kind=="no-model-file":
        relabel = True
    splitter = {split: SplitterWithData(iv,
                    dataset,
                    splits_path,
                    classes,
                    split_num,
                    split,
                    relabel) for split in ["train", "val", "test"]}
    loaders = {split: DataLoader(
                        splitter[split],
                        batch_size = batch_size,
                        drop_last = False,
                        shuffle = True)
                    for split in ["train", "val", "test"]}
    channel_idx = None    
    print("DONE: Create loaders for model")            
    return dataset, loaders, splitter

In [10]:
# EEGNet network
opt.classifier = "EEGNet"
opt.batch_size = 16
opt.kind = "from-scratch"
opt.run = "imagenet40-1000"
opt.fold = 5

In [11]:
dataset, loaders, splitter = load_dataset(opt.iv,
                             opt.offset,
                             opt.eeg_dataset,
                             opt.splits_path,
                             0, #split_num
                             n_classes,
                             classes,
                             opt.classifier,
                             opt.batch_size,
                             opt.GPUindex,
                             length,
                             channel,
                             min_CNN,
                             opt,
                             opt.kind)

DONE: LOAD DATASET
DONE: Create loaders for model


In [12]:
# loaders: divide the splits data in each fold with batch_size
# Each fold has {train: 32000 idx, val: 4000 idx, test: 4000 idx}
# Each loader batch has {train: 2000 idx, val: 250idx, test: 250 idx}
print(type(dataset))
print(type(loaders))
print(len(loaders), [len(loaders[name]) for name in ["train", "val", "test"] ])
for i in range(0, 40):
    eeg, label_val = splitter["val"][i*100]
    eeg, label_train = splitter["train"][i*800]
    print(f"{i+1}: Label val: {label_val}; label train: {label_train}")
# print(splitter["val"].split_idx[:10])

<class 'data_loader.EEGDataset'>
<class 'dict'>
3 [2000, 250, 250]
1: Label val: 0; label train: 0
2: Label val: 17; label train: 17
3: Label val: 28; label train: 28
4: Label val: 7; label train: 7
5: Label val: 33; label train: 33
6: Label val: 12; label train: 12
7: Label val: 21; label train: 21
8: Label val: 3; label train: 3
9: Label val: 25; label train: 25
10: Label val: 36; label train: 36
11: Label val: 10; label train: 10
12: Label val: 15; label train: 15
13: Label val: 19; label train: 19
14: Label val: 31; label train: 31
15: Label val: 23; label train: 23
16: Label val: 5; label train: 5
17: Label val: 38; label train: 38
18: Label val: 8; label train: 8
19: Label val: 1; label train: 1
20: Label val: 34; label train: 34
21: Label val: 29; label train: 29
22: Label val: 26; label train: 26
23: Label val: 13; label train: 13
24: Label val: 11; label train: 11
25: Label val: 22; label train: 22
26: Label val: 18; label train: 18
27: Label val: 6; label train: 6
28: Label v

In [13]:
net, nonclasses = Classifier(
                 n_classes,
                 classes,
                 opt.classifier,
                 opt.GPUindex,
                 length,
                 channel,
                 min_CNN,
                 opt.kind)
# print(len(nonclasses))
summary(net, input_size=(1,96, 512))

DONE: CREATE TORCH CLASSIFIER
classifier_EEGNet(
  (network): Sequential(
    (0): ZeroPad2d((128, 127, 0, 0))
    (1): Conv2d(1, 8, kernel_size=(1, 256), stride=(1, 1))
    (2): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(8, 16, kernel_size=(96, 1), stride=(1, 1), groups=8)
    (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ELU(alpha=1.0)
    (6): AvgPool2d(kernel_size=(1, 4), stride=(1, 4), padding=0)
    (7): Dropout(p=0.5, inplace=False)
    (8): ZeroPad2d((8, 7, 0, 0))
    (9): Conv2d(16, 16, kernel_size=(1, 16), stride=(1, 1), groups=8)
    (10): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1))
    (11): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ELU(alpha=1.0)
    (13): AvgPool2d(kernel_size=(1, 8), stride=(1, 8), padding=0)
    (14): Dropout(p=0.5, inplace=False)
  )
  (fc): Linear(in_features=256, out_features=40, bias=True)
)


Layer (type:depth-idx)                   Output Shape              Param #
classifier_EEGNet                        [1, 40]                   --
├─Sequential: 1-1                        [1, 16, 1, 16]            --
│    └─ZeroPad2d: 2-1                    [1, 1, 96, 767]           --
│    └─Conv2d: 2-2                       [1, 8, 96, 512]           2,056
│    └─BatchNorm2d: 2-3                  [1, 8, 96, 512]           16
│    └─Conv2d: 2-4                       [1, 16, 1, 512]           1,552
│    └─BatchNorm2d: 2-5                  [1, 16, 1, 512]           32
│    └─ELU: 2-6                          [1, 16, 1, 512]           --
│    └─AvgPool2d: 2-7                    [1, 16, 1, 128]           --
│    └─Dropout: 2-8                      [1, 16, 1, 128]           --
│    └─ZeroPad2d: 2-9                    [1, 16, 1, 143]           --
│    └─Conv2d: 2-10                      [1, 16, 1, 128]           528
│    └─Conv2d: 2-11                      [1, 16, 1, 128]           272
│    └─

In [14]:
model_path = (opt.iv+
                  "-"+
                  opt.classifier+
                  "-"+
                  str(length)+
                  "-"+
                  str(channel)+
                  "-"+
                  str(0)) + "-noshuffle"
                  
channel_idx=None
print(model_path)

image-EEGNet-512-96-0-noshuffle


In [16]:
print(opt)

Namespace(iv='image', offset=None, results_file='results.pkl', subject=0, run='imagenet40-1000', eeg_dataset='/media/titan/AI Research/Data/CVPR2021-02785/CVPR2021-02785/preprocessed/torch_models/imagenet40-1000-1.pth', splits_path='/media/titan/AI Research/Data/CVPR2021-02785/CVPR2021-02785/preprocessed/torch_models/imagenet40-1000-1_splits.pth', fold=5, batch_size=16, optim='Adam', learning_rate=0.001, learning_rate_decay_by=0.5, learning_rate_decay_every=10, epochs=100, GPUindex=0, kind='from-scratch', no_cuda=False, classifier='EEGNet')


In [17]:
if opt.kind=="from-scratch":
    loss_history, accuracy_val, accuracy_test = net_trainer(
            net,
            loaders,
            opt,
            channel_idx,
            nonclasses,
            None,
            True,
            model_path)

epoch 1
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 1000: Loss=3.7347614765167236; accuracy=0.027562500908970833
Batch 2000: Loss=3.7461633682250977; accuracy=0.02656250074505806
Validation accuracy: 0.03425000235438347
epoch 2
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 1000: Loss=3.6362812519073486; accuracy=0.036125000566244125
Batch 2000: Loss=3.6209020614624023; accuracy=0.037437502294778824
Validation accuracy: 0.04375000298023224
epoch 3
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 1000: Loss=3.7041988372802734; accuracy=0.04018750041723251
Batch 2000: Loss=3.551424264907837; accuracy=0.040687501430511475
Validation accuracy: 0.04325000196695328
epoch 4
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size(

Validation accuracy: 0.05975000187754631
epoch 29
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 1000: Loss=3.4342727661132812; accuracy=0.06356250494718552
Batch 2000: Loss=3.5470194816589355; accuracy=0.06400000303983688
Validation accuracy: 0.055000003427267075
epoch 30
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 1000: Loss=3.791875123977661; accuracy=0.06850000470876694
Batch 2000: Loss=3.3184633255004883; accuracy=0.06534375250339508
Validation accuracy: 0.058250002562999725
epoch 31
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 1000: Loss=3.580789804458618; accuracy=0.06931250542402267
Batch 2000: Loss=3.5937108993530273; accuracy=0.06737500429153442
Validation accuracy: 0.05900000408291817
epoch 32
Size of input:  torch.Size([16, 96, 512])
Size of target:  tor

Batch 2000: Loss=3.3608157634735107; accuracy=0.06843750178813934
Validation accuracy: 0.06050000339746475
epoch 57
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 1000: Loss=3.6988608837127686; accuracy=0.06931250542402267
Batch 2000: Loss=3.3499255180358887; accuracy=0.06800000369548798
Validation accuracy: 0.0625
epoch 58
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 1000: Loss=3.666477680206299; accuracy=0.07150000333786011
Batch 2000: Loss=3.6666533946990967; accuracy=0.07009375095367432
Validation accuracy: 0.058000002056360245
epoch 59
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 1000: Loss=3.748046636581421; accuracy=0.06781250238418579
Batch 2000: Loss=3.741029739379883; accuracy=0.06665625423192978
Validation accuracy: 0.05450000241398811
epoch 60
Size of inp

Batch 2000: Loss=3.702512741088867; accuracy=0.06921875476837158
Validation accuracy: 0.061250001192092896
epoch 85
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 1000: Loss=3.5404927730560303; accuracy=0.07331250607967377
Batch 2000: Loss=3.6607563495635986; accuracy=0.07040625065565109
Validation accuracy: 0.057500001043081284
epoch 86
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 1000: Loss=3.4906582832336426; accuracy=0.0715625062584877
Batch 2000: Loss=3.4337708950042725; accuracy=0.07046875357627869
Validation accuracy: 0.058000002056360245
epoch 87
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 1000: Loss=3.5095365047454834; accuracy=0.07218750566244125
Batch 2000: Loss=3.62040114402771; accuracy=0.07100000232458115
Validation accuracy: 0.05975000187754631
epoch 

In [18]:
val =accuracy_val
test = accuracy_test

print("Validation accuracy: ", val)
print("Test accuracy: ", test)

Validation accuracy:  0.0572500042617321
Test accuracy:  0


In [ ]:
print(accuracy_val)